In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import gmaps
import gmaps.datasets

# Import API key
from api_keys import gkey

In [63]:
# Store Part I results into DataFrame

# Load the csv exported in Part I to a DataFrame

In [64]:
# Store csv created in part one into  DataFrame
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tuktoyaktuk,69.4541,-133.0374,34.30,95,97,10.40,CA,1651374730
1,1,rikitea,-23.1203,-134.9692,76.78,73,96,9.91,PF,1651374730
2,2,upernavik,72.7868,-56.1549,-0.72,75,7,7.07,GL,1651374731
3,3,kloulklubed,7.0419,134.2556,82.90,76,90,12.17,PW,1651374731
4,4,mount isa,-20.7333,139.5000,82.17,39,40,14.97,AU,1651374732


In [65]:
# # # Humidity Heatmap

# # Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# AAdd Heatmap layer to map.

In [66]:
# Configure gmaps
gmaps.configure(api_key=gkey)

In [67]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [68]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.


In [69]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
110,110,saint george,37.1041,-113.5841,79.39,11,0,6.91,US,1651374521
252,252,santa ana,33.7456,-117.8678,73.71,58,0,8.05,US,1651374819
337,337,saint-pierre,-21.3393,55.4781,74.88,74,0,8.05,RE,1651374857
412,412,callaway,30.1530,-85.5699,72.91,92,0,9.22,US,1651374957
540,540,houma,29.5958,-90.7195,75.20,89,0,6.91,US,1651374875


In [70]:
# #  Hotel Map

# # Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame
# .Plot markers on top of the heatmap.

In [71]:
 # Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
110,saint george,US,37.1041,-113.5841,
252,santa ana,US,33.7456,-117.8678,
337,saint-pierre,RE,-21.3393,55.4781,
412,callaway,US,30.1530,-85.5699,
540,houma,US,29.5958,-90.7195,


In [72]:
hotel_df["Hotel Name"].count()

5

In [73]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
110,saint george,US,37.1041,-113.5841,
252,santa ana,US,33.7456,-117.8678,
337,saint-pierre,RE,-21.3393,55.4781,
412,callaway,US,30.1530,-85.5699,
540,houma,US,29.5958,-90.7195,


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [75]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))